# Plant Pathology 2021 - FGVC8: Plan

Objectives:
- Verify GPU availability and environment.
- Inspect data schema (train.csv, sample_submission.csv) and image directories.
- Establish robust CV and baseline image model (torch/TF) with multilabel micro-F1.
- Produce a working submission quickly, then iterate on aug/resolution/architectures and ensembling.

Initial Plan:
1) Env check (GPU).
2) Load train.csv/test images; verify multilabel format and submission format.
3) Quick EDA: label distribution, any leaks, filename mapping.
4) Baseline: torchvision + timm (e.g., tf_efficientnet_b0/b3, resnet50), resolution 512, strong aug (albumentations), BCEWithLogitsLoss, sigmoid threshold tuning on CV.
5) CV: StratifiedKFold for multilabel via iterative stratification (skmultilearn/iterstrat) or multilabel stratifier; otherwise GroupKFold if groups exist.
6) Logging: per-fold micro-F1, OOF saving, test-time augmentation, threshold calibration.
7) Submit baseline; iterate with higher res, different backbones, seeds, and blends.

Checkpoints for Expert Review:
- After env/EDA, after baseline CV setup, after first baseline OOF, after improvements/ensembles.

In [ ]:
import os, subprocess, time, glob, sys
import pandas as pd

def run(cmd):
    print("$", " ".join(cmd), flush=True)
    try:
        out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, check=False)
        print(out.stdout)
    except Exception as e:
        print("Command failed:", e)

print("Env check: nvidia-smi")
run(['bash','-lc','nvidia-smi || true'])

base = os.getcwd()
print("CWD:", base)
print("Listing top-level files:")
for p in sorted(os.listdir(base)):
    print(" -", p)

train_csv = 'train.csv'
sample_csv = 'sample_submission.csv' if os.path.exists('sample_submission.csv') else None
print("\nLoading CSVs...")
train_df = pd.read_csv(train_csv)
print("train.csv shape:", train_df.shape)
print(train_df.head(3))
if sample_csv:
    sample_df = pd.read_csv(sample_csv)
    print("sample_submission.csv shape:", sample_df.shape)
    print(sample_df.head(3))

train_images_dir = 'train_images'
test_images_dir = 'test_images'
train_imgs = glob.glob(os.path.join(train_images_dir, '*'))
test_imgs = glob.glob(os.path.join(test_images_dir, '*'))
print(f"Train images: {len(train_imgs)} | Test images: {len(test_imgs)}")
if len(train_imgs)>0:
    print("Sample train images:", [os.path.basename(p) for p in train_imgs[:5]])
if len(test_imgs)>0:
    print("Sample test images:", [os.path.basename(p) for p in test_imgs[:5]])

print("\nColumns in train.csv:", list(train_df.columns))
print("Done env/data probe.")

In [1]:
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Uninstall any preinstalled torch stack to avoid mismatches
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install exact CUDA 12.1 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze versions
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# 3) Install non-torch deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9',
    'albumentations==1.4.14',
    'opencv-python-headless',
    'iterative-stratification',
    'scikit-learn',
    'pandas', 'numpy',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity check GPU
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

Found existing installation: torch 2.4.1+cu121


Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121


Found existing installation: torchvision 0.19.1+cu121
Uninstalling torchvision-0.19.1+cu121:
  Successfully uninstalled torchvision-0.19.1+cu121


Found existing installation: torchaudio 2.4.1+cu121
Uninstalling torchaudio-2.4.1+cu121:
  Successfully uninstalled torchaudio-2.4.1+cu121
> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 180.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 233.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 400.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 477.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 492.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 461.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 519.1 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


In [ ]:
import os, json, numpy as np, pandas as pd
from collections import Counter
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Reload train to be safe in this cell context
train_df = pd.read_csv('train.csv')

# Parse space-delimited multilabels
label_lists = train_df['labels'].astype(str).str.strip().str.split()
all_labels = [lab for labs in label_lists for lab in labs]
label_counts = Counter(all_labels)
classes = sorted(label_counts.keys())  # preserve alpha order; alternative: sort by freq
print('Num classes:', len(classes))
print('Classes:', classes)
print('Top counts:', label_counts.most_common(10))

# Multi-hot encode
cls2id = {c:i for i,c in enumerate(classes)}
y = np.zeros((len(train_df), len(classes)), dtype=np.uint8)
for i, labs in enumerate(label_lists):
    for lab in labs:
        y[i, cls2id[lab]] = 1

# Save class list for reuse
with open('classes.json','w') as f:
    json.dump({'classes': classes}, f)
print('Saved classes.json')

# 5-fold Multilabel Stratified CV
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = np.full(len(train_df), -1, dtype=int)
for fold, (_, val_idx) in enumerate(mskf.split(train_df['image'].values, y)):
    folds[val_idx] = fold
assert (folds>=0).all(), 'Fold assignment failed'

train_folds = train_df.copy()
train_folds['fold'] = folds
train_folds.to_csv('train_folds.csv', index=False)
print('Saved train_folds.csv with fold distribution:')
print(train_folds['fold'].value_counts().sort_index())

# Basic sanity: label distribution per fold
for f in range(5):
    idx = (folds==f)
    cnt = y[idx].sum(axis=0)
    print(f'Fold {f}: n={idx.sum()} | positive labels total={int(cnt.sum())}')
print('CV setup complete.')

In [ ]:
import os, time, math, json, random, gc
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import timm
from timm.utils import ModelEmaV2
from timm.loss import AsymmetricLossMultiLabel
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
if hasattr(torch.backends, 'cuda') and hasattr(torch.backends.cuda, 'matmul'):
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_fp16_reduced_precision_reduction = True
    torch.backends.cudnn.allow_tf32 = True
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = os.environ.get('PYTORCH_CUDA_ALLOC_CONF','') or 'expandable_segments:True'
cv2.setNumThreads(0)

# Config
IMG_SIZE = 512
BATCH_SIZE = 10
EPOCHS = 15  # per expert, train up to 15 with early stopping
LR = 3e-4
WD = 1e-2
NUM_FOLDS = 5  # full CV
MODEL_NAME = 'tf_efficientnetv2_m.in21k'
DROP_PATH = 0.2
DROP_RATE = 0.05
TRAIN_DIR = 'train_images'
TEST_DIR = 'test_images'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DO_TRAIN = True  # train full 5-folds

# Performance/stability toggles
USE_CHANNELS_LAST = True
USE_BF16_AMP = True
USE_GRAD_CKPT = False
USE_EMA = True
EMA_DECAY = 0.9998

# Mixup config (manual for multilabel)
MIXUP_ALPHA = 0.4
MIXUP_PROB = 0.5  # will be turned off for last 2 epochs

# Defaults; will be overridden per model via timm.data.resolve_model_data_config(model)
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)
INTERP = cv2.INTER_CUBIC

# Load metadata
train_folds = pd.read_csv('train_folds.csv')
train_df = pd.read_csv('train.csv')
with open('classes.json') as f:
    classes = json.load(f)['classes']
C = len(classes)
cls2id = {c:i for i,c in enumerate(classes)}

# Parse labels to multi-hot
def labels_to_multi_hot(s):
    arr = np.zeros(C, dtype=np.float32)
    for lab in str(s).strip().split():
        if lab in cls2id:
            arr[cls2id[lab]] = 1.0
    return arr
y_all = np.stack(train_df.labels.apply(labels_to_multi_hot).values)

# Simple CV2-based transforms with RRC-like crop and Random Erasing
class SimpleTransform:
    def __init__(self, train=True, img_size=448, mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225), interp=cv2.INTER_CUBIC,
                 rrc_scale=(0.8, 1.0), rrc_ratio=(0.75, 1.3333),
                 erase_p=0.2, erase_area=(0.02, 0.2)):
        self.train = train
        self.img_size = img_size
        self.mean = np.array(mean, dtype=np.float32)
        self.std = np.array(std, dtype=np.float32)
        self.interp = interp
        self.rrc_scale = rrc_scale
        self.rrc_ratio = rrc_ratio
        self.erase_p = erase_p
        self.erase_area = erase_area

    def random_resized_crop(self, img):
        h, w = img.shape[:2]
        area = h * w
        for _ in range(10):
            target_area = area * random.uniform(self.rrc_scale[0], self.rrc_scale[1])
            aspect = random.uniform(self.rrc_ratio[0], self.rrc_ratio[1])
            new_w = int(round(math.sqrt(target_area * aspect)))
            new_h = int(round(math.sqrt(target_area / aspect)))
            if new_w <= w and new_h <= h and new_w > 0 and new_h > 0:
                x0 = random.randint(0, w - new_w)
                y0 = random.randint(0, h - new_h)
                return img[y0:y0+new_h, x0:x0+new_w]
        # Fallback to center crop
        min_side = min(h, w)
        y0 = (h - min_side) // 2
        x0 = (w - min_side) // 2
        return img[y0:y0+min_side, x0:x0+min_side]

    def random_erasing(self, img):
        # img is float32 normalized HWC
        if random.random() >= self.erase_p:
            return img
        h, w = img.shape[:2]
        area = h * w
        for _ in range(10):
            erase_area = area * random.uniform(self.erase_area[0], self.erase_area[1])
            aspect = random.uniform(0.3, 3.3)
            eh = int(round(math.sqrt(erase_area / aspect)))
            ew = int(round(math.sqrt(erase_area * aspect)))
            if eh <= h and ew <= w and eh > 0 and ew > 0:
                y0 = random.randint(0, h - eh)
                x0 = random.randint(0, w - ew)
                # Fill with mean color (0 mean in normalized space isn't correct; use dataset mean/std to approximate original mean=0)
                fill = np.zeros((eh, ew, 3), dtype=img.dtype)
                img[y0:y0+eh, x0:x0+ew, :] = fill
                return img
        return img

    def __call__(self, img):
        if self.train:
            img = self.random_resized_crop(img)
            if random.random() < 0.5:
                img = cv2.flip(img, 1)
            if random.random() < 0.2:
                img = cv2.flip(img, 0)
            if random.random() < 0.3:
                angle = random.uniform(-15, 15)
                M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), angle, 1.0)
                img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=self.interp, borderMode=cv2.BORDER_REFLECT_101)
            if random.random() < 0.2:
                hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.int32)
                hsv[...,0] = np.clip(hsv[...,0] + random.randint(-5,5), 0, 179)
                hsv[...,1] = np.clip(hsv[...,1] + random.randint(-10,10), 0, 255)
                hsv[...,2] = np.clip(hsv[...,2] + random.randint(-10,10), 0, 255)
                img = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)
            if random.random() < 0.1:
                k = random.choice([3,5])
                img = cv2.GaussianBlur(img, (k,k), 0)
        # resize, normalize
        img = cv2.resize(img, (self.img_size, self.img_size), interpolation=self.interp)
        img = img.astype(np.float32) / 255.0
        img = (img - self.mean) / self.std
        # random erasing after norm
        if self.train:
            img = self.random_erasing(img)
        return img

def get_transforms(train=True):
    return SimpleTransform(train=train, img_size=IMG_SIZE, mean=MEAN, std=STD, interp=INTERP)

class PlantDataset(Dataset):
    def __init__(self, df, labels=None, img_dir=TRAIN_DIR, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image'])
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img.transpose(2,0,1)).float()
        if self.labels is not None:
            target = self.labels[idx]
            return img, torch.from_numpy(target).float()
        else:
            return img, row['image']

def micro_f1(y_true, y_prob, thresh=0.3):
    y_pred = (y_prob >= thresh).astype(np.int32)
    if (y_pred.sum(axis=1)==0).any():
        for i in np.where(y_pred.sum(axis=1)==0)[0]:
            y_pred[i, y_prob[i].argmax()] = 1
    if 'healthy' in cls2id:
        h = cls2id['healthy']
        disease_idx = [i for i,c in enumerate(classes) if c!='healthy']
        disease_on = (y_pred[:, disease_idx].sum(axis=1) > 0)
        y_pred[disease_on, h] = 0
    return f1_score(y_true.ravel(), y_pred.ravel(), average='micro')

def micro_f1_vec(y_true, y_prob, thrs):
    y_pred = (y_prob >= thrs[None, :]).astype(np.int32)
    if (y_pred.sum(axis=1)==0).any():
        for i in np.where(y_pred.sum(axis=1)==0)[0]:
            y_pred[i, y_prob[i].argmax()] = 1
    if 'healthy' in cls2id:
        h = cls2id['healthy']
        disease_idx = [i for i,c in enumerate(classes) if c!='healthy']
        disease_on = (y_pred[:, disease_idx].sum(axis=1) > 0)
        y_pred[disease_on, h] = 0
    return f1_score(y_true.ravel(), y_pred.ravel(), average='micro')

def tune_global_threshold(y_true, y_prob, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.6, 12)
    best_t, best_f1 = 0.3, -1
    for t in grid:
        f1 = micro_f1(y_true, y_prob, t)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

def tune_thresholds_coordinate_descent(y_true, y_prob, base_thr=0.5, grid=None, iters=2):
    if grid is None:
        grid = np.linspace(0.05, 0.8, 31)  # ~0.025 step
    thrs = np.full(y_prob.shape[1], base_thr, dtype=np.float32)
    best = micro_f1_vec(y_true, y_prob, thrs)
    for _ in range(iters):
        improved = False
        for c in range(y_prob.shape[1]):
            best_c_thr = thrs[c]
            best_c_f1 = best
            for t in grid:
                thrs_try = thrs.copy(); thrs_try[c] = t
                f1 = micro_f1_vec(y_true, y_prob, thrs_try)
                if f1 > best_c_f1:
                    best_c_f1 = f1; best_c_thr = t
            if best_c_thr != thrs[c]:
                thrs[c] = best_c_thr
                best = best_c_f1
                improved = True
        if not improved:
            break
    # clamp to avoid extreme overfit
    thrs = np.clip(thrs, 0.05, 0.80).astype(np.float32)
    return thrs, best

def train_one_fold(fold):
    global MEAN, STD, INTERP
    t0 = time.time()
    print(f'===== Fold {fold} start =====')
    trn_idx = train_folds.index[train_folds['fold'] != fold].values
    val_idx = train_folds.index[train_folds['fold'] == fold].values
    df_trn = train_folds.iloc[trn_idx][['image']].reset_index(drop=True)
    df_val = train_folds.iloc[val_idx][['image']].reset_index(drop=True)
    y_trn = y_all[trn_idx]
    y_val = y_all[val_idx]

    # Model & data config
    model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=C, drop_path_rate=DROP_PATH, drop_rate=DROP_RATE)
    data_cfg = timm.data.resolve_model_data_config(model)
    MEAN, STD = tuple(data_cfg.get('mean', MEAN)), tuple(data_cfg.get('std', STD))
    interp_name = str(data_cfg.get('interpolation', 'bicubic')).lower()
    INTERP = cv2.INTER_CUBIC if 'bicubic' in interp_name else cv2.INTER_LINEAR

    if USE_GRAD_CKPT and hasattr(model, 'set_grad_checkpointing'):
        try:
            model.set_grad_checkpointing(True)
            print('Enabled gradient checkpointing')
        except Exception:
            pass
    model.to(DEVICE)
    if USE_CHANNELS_LAST:
        model.to(memory_format=torch.channels_last)

    ema = ModelEmaV2(model, decay=EMA_DECAY, device=DEVICE) if USE_EMA else None

    # Datasets now that MEAN/STD/INTERP are set
    train_ds = PlantDataset(df_trn, y_trn, img_dir=TRAIN_DIR, transform=get_transforms(True))
    val_ds = PlantDataset(df_val, y_val, img_dir=TRAIN_DIR, transform=get_transforms(False))
    nw = min(8, os.cpu_count() or 4)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=nw, pin_memory=True, drop_last=True, persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=nw, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, eps=1e-8, betas=(0.9,0.999))
    warmup_steps = max(1, len(train_loader))
    total_steps = max(warmup_steps+1, EPOCHS * len(train_loader))
    sched_warm = LinearLR(opt, start_factor=0.01, total_iters=warmup_steps)
    sched_cos = CosineAnnealingLR(opt, T_max=max(1, total_steps - warmup_steps))
    scheduler = SequentialLR(opt, schedulers=[sched_warm, sched_cos], milestones=[warmup_steps])
    criterion = AsymmetricLossMultiLabel(gamma_neg=4.0, gamma_pos=0.0, clip=0.05, eps=1e-8)

    best_f1 = -1.0
    best_path = f'model_fold{fold}.pt'
    patience = 3
    bad_epochs = 0

    for epoch in range(EPOCHS):
        model.train()
        t_ep = time.time()
        # decay Mixup prob to 0 in last 2 epochs
        mixup_prob_now = MIXUP_PROB if epoch < EPOCHS - 2 else 0.0
        for it, (imgs, targets) in enumerate(train_loader):
            if USE_CHANNELS_LAST:
                imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
            else:
                imgs = imgs.to(DEVICE, non_blocking=True)
            targets = targets.to(DEVICE, non_blocking=True)
            # Manual Mixup for multilabel
            if MIXUP_ALPHA > 0 and random.random() < mixup_prob_now:
                lam = float(np.random.beta(MIXUP_ALPHA, MIXUP_ALPHA))
                idx = torch.randperm(imgs.size(0), device=imgs.device)
                imgs = lam * imgs + (1.0 - lam) * imgs[idx]
                targets = lam * targets + (1.0 - lam) * targets[idx]
            opt.zero_grad(set_to_none=True)
            if USE_BF16_AMP and DEVICE=='cuda':
                with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                    logits = model(imgs)
                    loss = criterion(logits.float(), targets)
            else:
                logits = model(imgs)
                loss = criterion(logits, targets)

            if not torch.isfinite(loss):
                print(f'Non-finite loss detected at iter {it}: {loss.item()} -> skipping step')
                opt.zero_grad(set_to_none=True)
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            scheduler.step()
            if ema is not None:
                ema.update(model)
            if it % 50 == 0:
                cur_lr = scheduler.get_last_lr()[0] if hasattr(scheduler, 'get_last_lr') else opt.param_groups[0]['lr']
                print(f'Fold {fold} Epoch {epoch} Iter {it}/{len(train_loader)} loss={loss.item():.4f} lr={cur_lr:.6f}')

        model.eval()
        eval_model = ema.module if ema is not None else model
        preds = []; gts = []
        with torch.no_grad():
            for imgs, targets in val_loader:
                if USE_CHANNELS_LAST:
                    imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                else:
                    imgs = imgs.to(DEVICE, non_blocking=True)
                if USE_BF16_AMP and DEVICE=='cuda':
                    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                        logits = eval_model(imgs)
                else:
                    logits = eval_model(imgs)
                preds.append(torch.sigmoid(logits.float()).cpu().numpy())
                gts.append(targets.cpu().numpy())
        y_prob = np.concatenate(preds, axis=0)
        y_true = np.concatenate(gts, axis=0)
        t_opt, f1_opt = tune_global_threshold(y_true, y_prob)
        print(f'Epoch {epoch} val micro-F1={f1_opt:.5f} @thr={t_opt:.3f} | time {time.time()-t_ep:.1f}s')
        improved = f1_opt > best_f1 + 1e-5
        if improved and np.isfinite(f1_opt):
            best_f1 = f1_opt
            torch.save({'model': eval_model.state_dict(), 'thr': t_opt}, best_path)
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print('Early stopping due to no improvement')
                break

    ckpt = torch.load(best_path, map_location=DEVICE)
    (ema.module if ema is not None else model).load_state_dict(ckpt['model'])
    thr = ckpt.get('thr', 0.3)
    (ema.module if ema is not None else model).eval()
    preds = []
    with torch.no_grad():
        for imgs, targets in val_loader:
            if USE_CHANNELS_LAST:
                imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
            else:
                imgs = imgs.to(DEVICE, non_blocking=True)
            if USE_BF16_AMP and DEVICE=='cuda':
                with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                    logits = (ema.module if ema is not None else model)(imgs)
            else:
                logits = (ema.module if ema is not None else model)(imgs)
            preds.append(torch.sigmoid(logits.float()).cpu().numpy())
    y_prob = np.concatenate(preds, axis=0)
    print(f'Fold {fold} done in {time.time()-t0:.1f}s, best_f1={best_f1:.5f}, thr={thr:.3f}')
    return y_prob, y_true, thr

def infer_test(models_paths, tta=2):
    global MEAN, STD, INTERP
    # Ensure normalization matches model cfg even in fresh kernels
    tmp_model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=C, drop_path_rate=DROP_PATH, drop_rate=DROP_RATE)
    data_cfg = timm.data.resolve_model_data_config(tmp_model)
    MEAN, STD = tuple(data_cfg.get('mean', MEAN)), tuple(data_cfg.get('std', STD))
    interp_name = str(data_cfg.get('interpolation', 'bicubic')).lower()
    INTERP = cv2.INTER_CUBIC if 'bicubic' in interp_name else cv2.INTER_LINEAR

    test_df = pd.read_csv('sample_submission.csv')[['image']].copy()
    ds = PlantDataset(test_df, labels=None, img_dir=TEST_DIR, transform=get_transforms(False))
    loader = DataLoader(ds, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=min(8, os.cpu_count() or 4), pin_memory=True, persistent_workers=True, prefetch_factor=2)
    model_level_logits = []  # list of (N_test, C)
    for mp in models_paths:
        model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=C, drop_path_rate=DROP_PATH, drop_rate=DROP_RATE)
        ckpt = torch.load(mp, map_location=DEVICE)
        model.load_state_dict(ckpt['model'])
        model.to(DEVICE)
        if USE_CHANNELS_LAST:
            model.to(memory_format=torch.channels_last)
        model.eval()
        view_logits = []  # per-TTA view logits (N_test, C)
        with torch.no_grad():
            # view 1: original
            outs = []
            for imgs, names in loader:
                if USE_CHANNELS_LAST:
                    imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                else:
                    imgs = imgs.to(DEVICE, non_blocking=True)
                if USE_BF16_AMP and DEVICE=='cuda':
                    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                        logits = model(imgs)
                else:
                    logits = model(imgs)
                outs.append(logits.float().cpu().numpy())
            view_logits.append(np.concatenate(outs, axis=0))
            # view 2: hflip
            if tta >= 2:
                outs = []
                for imgs, names in loader:
                    imgs = imgs.flip(-1)
                    if USE_CHANNELS_LAST:
                        imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                    else:
                        imgs = imgs.to(DEVICE, non_blocking=True)
                    if USE_BF16_AMP and DEVICE=='cuda':
                        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                            logits = model(imgs)
                    else:
                        logits = model(imgs)
                    outs.append(logits.float().cpu().numpy())
                view_logits.append(np.concatenate(outs, axis=0))
            # view 3: vflip
            if tta >= 3:
                outs = []
                for imgs, names in loader:
                    imgs = imgs.flip(-2)
                    if USE_CHANNELS_LAST:
                        imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                    else:
                        imgs = imgs.to(DEVICE, non_blocking=True)
                    if USE_BF16_AMP and DEVICE=='cuda':
                        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                            logits = model(imgs)
                    else:
                        logits = model(imgs)
                    outs.append(logits.float().cpu().numpy())
                view_logits.append(np.concatenate(outs, axis=0))
            # view 4: hvflip
            if tta >= 4:
                outs = []
                for imgs, names in loader:
                    imgs = imgs.flip(-1).flip(-2)
                    if USE_CHANNELS_LAST:
                        imgs = imgs.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
                    else:
                        imgs = imgs.to(DEVICE, non_blocking=True)
                    if USE_BF16_AMP and DEVICE=='cuda':
                        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                            logits = model(imgs)
                    else:
                        logits = model(imgs)
                    outs.append(logits.float().cpu().numpy())
                view_logits.append(np.concatenate(outs, axis=0))
        logits_avg = np.mean(np.stack(view_logits, axis=0), axis=0)  # (N_test, C)
        model_level_logits.append(logits_avg)
    logits = np.mean(np.stack(model_level_logits, axis=0), axis=0)  # (N_test, C)
    probs = 1/(1+np.exp(-logits))
    return test_df['image'].values, probs

# Orchestrate K-fold training or skip to inference
if DO_TRAIN:
    oof_probs = np.zeros((len(train_df), C), dtype=np.float32)
    oof_targets = y_all.copy()
    fold_thresholds = []
    for fold in range(NUM_FOLDS):
        t_fold = time.time()
        y_prob, y_true, thr = train_one_fold(fold)
        val_idx = train_folds.index[train_folds['fold'] == fold].values
        oof_probs[val_idx] = y_prob
        fold_thresholds.append(thr)
        print(f'Fold {fold} completed in {time.time()-t_fold:.1f}s')
        gc.collect(); torch.cuda.empty_cache()

    np.save('oof_probs.npy', oof_probs)
    np.save('oof_targets.npy', oof_targets)
    print('Saved OOF probs/targets')
    mask = train_folds['fold'].isin(list(range(NUM_FOLDS))).values
    t_best, f1_best = tune_global_threshold(oof_targets[mask], oof_probs[mask])
    # Per-class threshold tuning and save
    thrs_vec, f1_best_vec = tune_thresholds_coordinate_descent(oof_targets[mask], oof_probs[mask], base_thr=t_best, grid=np.linspace(0.05,0.8,31), iters=3)
    thrs_vec = np.clip(thrs_vec, 0.05, 0.80).astype(np.float32)
    np.save('thr_per_class.npy', thrs_vec)
    print(f'OOF (folds< {NUM_FOLDS}) micro-F1={f1_best:.5f} @thr={t_best:.3f}; per-class tuned micro-F1={f1_best_vec:.5f} | n={mask.sum()}')
else:
    if os.path.exists('oof_probs.npy') and os.path.exists('oof_targets.npy'):    
        oof_probs = np.load('oof_probs.npy')
        oof_targets = np.load('oof_targets.npy')
        mask = train_folds['fold'].isin(list(range(NUM_FOLDS))).values
        t_best, f1_best = tune_global_threshold(oof_targets[mask], oof_probs[mask])
        thrs_vec, f1_best_vec = tune_thresholds_coordinate_descent(oof_targets[mask], oof_probs[mask], base_thr=t_best, grid=np.linspace(0.05,0.8,31), iters=3)
        thrs_vec = np.clip(thrs_vec, 0.05, 0.80).astype(np.float32)
        np.save('thr_per_class.npy', thrs_vec)
        print(f'Loaded OOF (folds< {NUM_FOLDS}); tuned global thr={t_best:.3f} (micro-F1={f1_best:.5f}); per-class tuned micro-F1={f1_best_vec:.5f} | n={mask.sum()}')
    else:
        t_best = 0.6
        thrs_vec = np.full(C, t_best, dtype=np.float32)
        print('OOF not found; using default thr=0.6')

# Inference on test using best fold checkpoints
model_paths = [f'model_fold{i}.pt' for i in range(NUM_FOLDS)]
names, test_probs = infer_test(model_paths, tta=4)

def probs_to_labels_row(p, thr_or_vec):
    if np.ndim(thr_or_vec)==0:
        lab_mask = (p >= float(thr_or_vec))
    else:
        lab_mask = (p >= thr_or_vec)
    if lab_mask.sum() == 0:
        lab_mask[p.argmax()] = 1
    if 'healthy' in cls2id:
        h = cls2id['healthy']
        disease_idx = [i for i,c in enumerate(classes) if c!='healthy']
        if lab_mask[disease_idx].sum() > 0:
            lab_mask[h] = 0
    return ' '.join([classes[i] for i in np.where(lab_mask)[0]])

thr_to_use = np.load('thr_per_class.npy') if os.path.exists('thr_per_class.npy') else t_best
labels_out = [probs_to_labels_row(p, thr_to_use) for p in test_probs]
sub = pd.DataFrame({'image': names, 'labels': labels_out})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with shape', sub.shape)

===== Fold 0 start =====


Fold 0 Epoch 0 Iter 0/1192 loss=47.3398 lr=0.000003


In [6]:
import sys, subprocess, os
def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Fix albumentations/albucore mismatch by pinning albumentations<1.4 (no albucore dependency)
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'albumentations', 'albucore'], check=False)
pip('install', 'albumentations==1.3.1', 'opencv-python-headless', '--upgrade-strategy', 'only-if-needed')
import albumentations as A
print('Albumentations loaded from:', getattr(A, '__file__', 'unknown'))

Found existing installation: albumentations 1.3.1
Uninstalling albumentations-1.3.1:
  Successfully uninstalled albumentations-1.3.1
> install albumentations==1.3.1 opencv-python-headless --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 KB 5.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 199.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 KB 504.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 401.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 138.2 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 190.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 KB 389.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 432.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 524.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 334.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 364.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 KB 530.1 MB/s eta 0:00:00


Albumentations loaded from: None


In [ ]:
# Quick near-duplicate scan via pHash (run separately from training).
import os, math, time, itertools
import numpy as np
import pandas as pd
import cv2

def phash64(img_bgr):
    # Convert to grayscale and compute 8x8 DCT-based perceptual hash (64-bit)
    try:
        img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
        img = np.float32(img)
        dct = cv2.dct(img)
        dct_low = dct[:8, :8].copy()
        dct_low[0,0] = 0.0  # remove DC
        med = np.median(dct_low)
        bits = (dct_low > med).astype(np.uint8).reshape(-1)
        # pack into 64-bit int
        h = 0
        for b in bits:
            h = (h << 1) | int(b)
        return np.uint64(h)
    except Exception:
        return np.uint64(0)

def hamming64(a, b):
    return int(bin(int(a ^ b)).count('1'))

def run_phash_scan(images_dir='train_images', max_bucket_size=200, prefix_bits=16, ham_thresh=5, sample=None):
    t0 = time.time()
    imgs = sorted(os.listdir(images_dir))
    if sample is not None and sample < len(imgs):
        imgs = imgs[:sample]
    paths = [os.path.join(images_dir, x) for x in imgs]
    hashes = []
    for i, p in enumerate(paths):
        im = cv2.imread(p)
        if im is None:
            hashes.append(np.uint64(0)); continue
        hashes.append(phash64(im))
        if i % 1000 == 0:
            print(f'pHash {i}/{len(paths)} processed')
    hashes = np.array(hashes, dtype=np.uint64)
    print('Computed hashes in', f'{time.time()-t0:.1f}s')

    # Exact duplicates (identical pHash)
    df = pd.DataFrame({'image': imgs, 'phash': hashes})
    dup_groups = df.groupby('phash').filter(lambda x: len(x) > 1)
    if len(dup_groups) > 0:
        print('Exact-duplicate pHash groups:', dup_groups.groupby('phash').size().shape[0])
    else:
        print('No exact-duplicate pHash groups found')

    # Approximate duplicates by prefix bucketing to limit pairwise work
    prefix_shift = 64 - prefix_bits
    prefixes = (hashes >> np.uint64(prefix_shift)).astype(np.uint64)
    buckets = {}
    for idx, pref in enumerate(prefixes):
        buckets.setdefault(int(pref), []).append(idx)
    print('Buckets:', len(buckets))

    pairs = []  # (img_a, img_b, ham)
    checked = 0
    for pref, idxs in buckets.items():
        if len(idxs) <= 1:
            continue
        if len(idxs) > max_bucket_size:
            # skip giant buckets to keep runtime bounded
            continue
        for i, j in itertools.combinations(idxs, 2):
            ham = hamming64(hashes[i], hashes[j])
            if ham <= ham_thresh:
                pairs.append((imgs[i], imgs[j], ham))
        checked += 1
        if checked % 200 == 0:
            print(f'Checked {checked}/{len(buckets)} buckets; pairs so far={len(pairs)}')

    dup_df = pd.DataFrame(pairs, columns=['image_a','image_b','hamming'])
    dup_df.to_csv('near_duplicate_pairs.csv', index=False)
    print('Saved near_duplicate_pairs.csv with', len(dup_df), 'pairs; total time', f'{time.time()-t0:.1f}s')

    # If folds exist, summarize cross-fold duplicates
    if os.path.exists('train_folds.csv'):
        folds = pd.read_csv('train_folds.csv')[['image','fold']]
        m = dup_df.merge(folds.rename(columns={'image':'image_a'}), on='image_a', how='left')
        m = m.merge(folds.rename(columns={'image':'image_b','fold':'fold_b'}), on='image_b', how='left')
        m = m.rename(columns={'fold':'fold_a'})
        m.to_csv('near_duplicate_pairs_with_folds.csv', index=False)
        if len(m):
            cross = (m['fold_a'] != m['fold_b']).mean()
            print(f'Cross-fold duplicate rate: {cross:.3f} over {len(m)} pairs')
        else:
            print('No near-duplicate pairs to summarize')

print('To run: run_phash_scan(images_dir="train_images", sample=None)')